# Ag-Analytics® - Elevation Index API 

Local elevation on a field can have a strong correlation with harvest yield. The correlation stems from the drainage dynamics that are present on a given field, with low areas being productive in dry years and high areas being productive in wet years. 

The Elevation Index API supports calculation of a set of elevation indices including field relative elevation, maximum slope, ,local Topographic Position Index(TPI), and local Topographic Roughness Index(TRI) for a given field elevation. 

### Required libraries

In [1]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameter Details

Request URL:  https://ag-analytics.azure-api.net/elevation-index
       
1) __Shape__(geojson string): Geojson boundary of the area of interest (AOI).       
       
2) __index_list__(string): The elevation index of interest to retrieve.
    
    i. Relative_Elevation: Derived by calculating the z-score of every point in the field
    
    ii. Slop: The maximum slope of each point on the field is calculated by taking the gradient of the elevation for each point.
    
    iii.TPI: The topographic position index is a method of measuring the elevation of a given point with respect to a surrounding neighborhood of surrounding points. This index indicates the local extrema,or points of maximum and minimum elevation on a sub-field level. 
    
    iV.TRI:  The topographic Roughness index express the amount of elevation difference between adjacent points on local field

3) __Outside_Radius(optional)__: Outer radius of search area of each point. Used in TPI only.

        (Ex:Outside_Radius =11 the outer search radius is 11 * cellsize. When cellsize is 0.001 of the elevation raster, the radius is about 11 * 8 = 88 meter

4) __Inside_Radius(optional)__: Inner radius of search area. Used in TPI only.

5) __Search_Radius(optional)__: Radius of computational box. search_radius = 1 => 3x3 box, 2 => 5x5 box, etc. Used in TRI only



### API Function

In [16]:
def ElevationIndex(values, headers):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
        
        response = requests.post(url, json=values, headers=headers).json()

        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

### Request with GeoJSON shape.

In [30]:
# Request as GeoJSON.
values = {'Shape': '{\"type\":\"Feature\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-76.5907145, 42.443918], [-76.5898132, 42.4224745], [-76.5699863, 42.4230447], [-76.5710592, 42.4443296], [-76.5907145, 42.443918]]]},\"properties\":{\"OBJECTID\":4944402,\"CALCACRES\":46.15999985,\"CALCACRES2\":null},\"id\":4944402}',
            'index_list':"['TRI']",
            'Outside_Radius' :'5',
            'Inside_Radius':'3',
            'Search_Radius' :'2',
            'Legend_Range': '10'
}

headers={'Content-Type':'application/json'}


In [31]:
IndexResponse = ElevationIndex(values, headers)

{'TRI': 'raster_TRI_20200622205918465442.tif', 'features': [[{'attributes': {'CellSize': [9.259259269220297e-05, -9.259259269220297e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-76.5907145, 42.42238515553195, -76.57006635182964, 42.4443296', 'Legend': [{'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': 0.4233935952186585, 'Mean': 0.23043658919632437, 'Min': 0.03747958317399025, 'color': '#4d76d5'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': 0.6059746384620667, 'Mean': 0.5146841168403626, 'Min': 0.4233935952186585, 'color': '#738bbd'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060, 'Max': 0.7470613121986389, 'Mean': 0.6765179753303527, 'Min': 0.6059746384620667, 'color': '#99a1a6'}, {'Area': '10.0 %', 'Count': 4906, 'CountAllPixels': 49060

### Output Index Summary

In [32]:
df=defaultdict(list)
for value in IndexResponse['features']:
    attributes = value[0]['attributes']
    df['Index'].append(attributes['Product'])
    df['Max'].append(float(attributes['Max']))
    df['Mean'].append(float(attributes['Mean']))
    df['Min'].append(float(attributes['Min']))
    df['pngb64'].append(attributes['pngb64'])

indexdf=pd.DataFrame.from_dict(df)
#indexdf = indexdf.sort_values(by = ['Area'], ascending=False).style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])
indexdf

,Index,Max,Mean,Min,pngb64
0,TRI,5.874935,1.215647,0.03748,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."


### Displaying Index Image

In [33]:
#####################################
# Select one index to display. Must match one of the following: 
# ['Slope', 'Relative_Elevation', 'TPI', 'TRI']
Index = 'TRI'
######################################

In [34]:
#Select one index
Index = Index

#Image Legend
df=defaultdict(list)
for product in IndexResponse['features']:
    if Index == product[0]['attributes']['Product']:
        Legend = product[0]['attributes']['Legend']
for val in Legend:
    df['Max'].append(float(val['Max']))
    df['Mean'].append(float(val['Mean']))
    df['Min'].append(float(val['Min']))
    df['Color'].append(val['color'])
    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])

,Max,Mean,Min,Color
0,0.423394,0.230437,0.037480,#4d76d5
1,0.605975,0.514684,0.423394,#738bbd
2,0.747061,0.676518,0.605975,#99a1a6
3,0.906737,0.826899,0.747061,#bfb68f
4,1.080329,0.993533,0.906737,#e5cc78
5,1.258940,1.169635,1.080329,#e5b470
6,1.452856,1.355898,1.258940,#e69c69
7,1.724798,1.588827,1.452856,#e68462
8,2.179848,1.952323,1.724798,#e76c5b
9,5.874935,4.027392,2.179848,#e85454


In [35]:
IndexImage = indexdf.loc[indexdf['Index'] == Index,'pngb64'].iloc[0]
Image(url = IndexImage, width= 500, height =500)

# GET Request

In [38]:
# Specify local path where file will be downloaded.
local_path = r"<Path to local directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"

# Create GET request payload.
Index = 'TRI'
values = {'filenames': [IndexResponse[Index],]}
print(values)

{'filenames': ['raster_TRI_20200622205918465442.tif']}


In [39]:
def Elevation_Index_get(values, local_path):
    try:
        url = 'https://ag-analytics.azure-api.net/elevation-index'
    
        download_path = os.path.join(local_path, values['filenames'][0])
        print(values)
     
        response = requests.get(url, params=values)
        open(download_path, 'wb').write(response.content)
        
        print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [40]:
Elevation_Index_get(values, local_path)

{'filenames': ['raster_TRI_20200622205918465442.tif']}
https://ag-analytics.azure-api.net/elevation-index?filenames=raster_TRI_20200622205918465442.tif


<Response [200]>